This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
!pip install --upgrade sentence-transformers huggingface_hub --quiet
!pip install langchain_openai --quiet
!pip install langchain_community --quiet
!pip install langchain_openai --quiet
!pip install langchain_chroma --quiet

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "voc-32034342212667733781206696e1b8b3b604.60383220"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

import langchain
import openai
import chromadb

### Generate random listings with gpt

In [3]:
# 1st method: for simple one line instructions

#user_prompt = "Generate 20 real estate listings with Neighborhood name, Price, Bedrooms, Bathrooms, House Size, House Description, Neighborhood Description, using a good format"
#model = "gpt-3.5-turbo-instruct"  #  to explicit instructions 
#response = openai.Completion.create(model = model, 
#                                    prompt=user_prompt,
#                                    max_tokens= 1500).choices[0].text.strip()

In [4]:
user_prompt = """
    Generate 10 real estate listings like this but with different data and descriptions, 
    the variety of the data can be broad but must have logic (for example more expensive for higher features). 
    Bathrooms can range from 1 to 2, Bedrooms can range from 1 to 4 as integers.
    House Size can range from 600 to 2500 sqft
    Besides, each listing must be separated by "\n==="

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Please generate ten (10) of such listings.
"""
model = "gpt-3.5-turbo" 

In [5]:
# 2nd method: to use multiline prompt (ideal for format output example BUT IT CREATES COMPATIBILITY PROBLEMS WITH LANGCHAIN.OPENAI)
#messages = []
#messages.append({"role":"system", "content":"you are an expert of real estate"})
#messages.append({"role": "user", "content": user_prompt})

#response = openai.ChatCompletion.create(model = model, 
#                                messages=messages,
#                                #max_tokens= 500,
#                                temperature = 1).choices[0].message['content']

In [6]:
from langchain_openai import ChatOpenAI

response = ChatOpenAI(
    model=model,
    temperature=1,
    max_tokens=3000,
    timeout=1000,
    max_retries=2
).invoke(user_prompt)
print(response.content)

Neighborhood: Lakeview Heights
Price: $650,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,200 sqft

Description: Enjoy panoramic views of the lake from this cozy 2-bedroom, 1-bathroom cottage in Lakeview Heights. This charming home features a spacious deck overlooking the water, perfect for morning coffee or evening sunsets. The open living area with a fireplace creates a warm and inviting atmosphere, ideal for relaxation or entertaining. With its serene setting and lake access, this home is a peaceful retreat from city life.

Neighborhood Description: Lakeview Heights is a picturesque lakeside community known for its tranquil and laid-back atmosphere. Residents can enjoy water activities such as boating and fishing, as well as hiking and biking trails in the surrounding natural areas. Local restaurants and cafes offer a taste of lakeside living, making it a desirable location for those seeking a peaceful getaway.

===
Neighborhood: Pinecrest Estates
Price: $1,200,000
Bedrooms: 4
Bathrooms

### Save documentation

In [7]:
# Save listings into txt
with open("Listings.txt", 'w') as file:
    file.write(response.content)

In [8]:
README = """
        The notebook creates a Real Estate Agent simulating 5 questions asked by AI and 5 answers given by User.
        At the start the OpenAI api is called to produce 10 fictional listings and are stored in a vector database.
        Then, the answers and questions are used to create the context to retrieve the best match from db 
        and reply to the user in a motivating manner.
        During the run you will see different methodologies of chaining to accomplish the task using langchain.
        The last one uses also memory to store the messages and retrieving them back 
        to use them as a context, this would offer presumably more flexibility.
        
        Dependencies installed:
        langchain
        huggingface_hub
        langchain_openai
        langchain_community
        langchain_openai
        langchain_chroma
        langchain_core
        
        Note: You may need to restart notebook twice for dependencies to be effective
        """
# Save documentation file into txt
with open("HomeMatchReadme.txt", 'w') as file:
    file.write(README)

### Create vector database

In [9]:
# Note: no need to use CharacterTextSplitter

text_list = response.content.split("===")

# let's see the first listing
print(text_list[0])

Neighborhood: Lakeview Heights
Price: $650,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,200 sqft

Description: Enjoy panoramic views of the lake from this cozy 2-bedroom, 1-bathroom cottage in Lakeview Heights. This charming home features a spacious deck overlooking the water, perfect for morning coffee or evening sunsets. The open living area with a fireplace creates a warm and inviting atmosphere, ideal for relaxation or entertaining. With its serene setting and lake access, this home is a peaceful retreat from city life.

Neighborhood Description: Lakeview Heights is a picturesque lakeside community known for its tranquil and laid-back atmosphere. Residents can enjoy water activities such as boating and fishing, as well as hiking and biking trails in the surrounding natural areas. Local restaurants and cafes offer a taste of lakeside living, making it a desirable location for those seeking a peaceful getaway.




In [10]:
from langchain.schema import Document

# Convert list of strings into Document objects
documents = [Document(page_content=text) for text in text_list]

In [11]:
from langchain_community.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/home/student/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
from langchain_chroma import Chroma #note: no need of lancedb because we don't have metadata like links of website or external related tables
db = Chroma.from_documents(documents, embeddings)

In [13]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0)

## User preferences

In [14]:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

#answers = []
#for question in questions:
#    answer = input("question")
#    context.append(zip(question, answers[question]))


# let's pretend user answered to all 5 questions
context = []
for i in range(len(questions)):
    context.append(questions[i])
    context.append(answers[i])
    
context = '\n'.join(context)
print(context)

How big do you want your house to be?What are 3 most important things for you in choosing this property?
A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
Which amenities would you like?
A quiet neighborhood, good local schools, and convenient shopping options.
Which transportation options are important to you?
A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
How urban do you want your neighborhood to be?
Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.


In [15]:
RECOMMENDER_TEMPLATE = """
    "The following is a conversation between a human and a AI Real Estate Agent"
    "You understand the preferences of the user and return the best option among the available listings"
    "If you don't know the answer related to house recommendation, say No matches are found in the available listings"
    "For other types of questions, if they are contextual to real estate, you can answer"
    "If the users ask off-topic questions out of real estate topic, say I don't know"
    "Use three sentences maximum and keep the answer concise."
    
    Personal Questions and Answers:
    {context}

    Human: {query}
    AI:
    """

### Recommender without memory

In [16]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA

# if questions are terminated starts the input (query) for the chain
query = "Find two apartments based on user preferences and after describing the listings giving all details, show the reason behind this suggestion"

# Prepare the formatted prompt
formatted_prompt = RECOMMENDER_TEMPLATE.format(context=context, query=query)
formatted_prompt

'\n    "The following is a conversation between a human and a AI Real Estate Agent"\n    "You understand the preferences of the user and return the best option among the available listings"\n    "If you don\'t know the answer related to house recommendation, say No matches are found in the available listings"\n    "For other types of questions, if they are contextual to real estate, you can answer"\n    "If the users ask off-topic questions out of real estate topic, say I don\'t know"\n    "Use three sentences maximum and keep the answer concise."\n    \n    Personal Questions and Answers:\n    How big do you want your house to be?What are 3 most important things for you in choosing this property?\nA comfortable three-bedroom house with a spacious kitchen and a cozy living room.\nWhich amenities would you like?\nA quiet neighborhood, good local schools, and convenient shopping options.\nWhich transportation options are important to you?\nA backyard for gardening, a two-car garage, and 

#### Not requiring external similarity search

###### -RetrievalQA chain

###### -Retrieval stuff documents chain

In [17]:
# 1st method
chain = RetrievalQA.from_chain_type(llm = llm,
                                    chain_type = "stuff", 
                                    retriever= db.as_retriever(search_kwargs={"k": 2}),
                                                                verbose=True,
                                                                )
result = chain.invoke(formatted_prompt)["result"]
print(result)



> Entering new RetrievalQA chain...

> Finished chain.
 Based on your preferences, I have found two apartments that may be suitable for you. The first is a charming 3-bedroom bungalow in Maple Grove with a spacious deck and private backyard, perfect for outdoor gatherings. The second is a cozy 3-bedroom home in Riverside Terrace with a fireplace and outdoor patio, ideal for a peaceful retreat by the river. Both listings offer the amenities you mentioned, such as a quiet neighborhood, good schools, and convenient shopping options. Additionally, they are both located in family-friendly communities with easy access to transportation options.


In [18]:
# 2nd method (requires BasePromptTemplate)

from langchain.prompts import PromptTemplate
from langchain.chains import create_history_aware_retriever

# Define the updated prompt template using 'input' and 'context'

    
# Update prompt template
prompt_template = PromptTemplate(
    template=RECOMMENDER_TEMPLATE,
    input_variables=["query", "context"]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt_template)
rag_chain = create_retrieval_chain(
                                    db.as_retriever(
                                                    search_type = 'similarity_score_threshold',
                                                    search_kwargs = {
                                                                    "k": 2,
                                                                    "score_threshold": 0.1
                                                    },)
                                   , question_answer_chain)

result = chain.invoke({"query": query, "context": context})["result"]
print(result)



> Entering new RetrievalQA chain...

> Finished chain.


Based on the user's preferences, I would suggest the first listing in Pineview Village. This charming 2-bedroom, 1-bathroom cottage offers a cozy and peaceful retreat in a quaint village setting. The outdoor space with a fire pit and detached studio adds to the appeal of this home, making it a perfect option for those seeking a home office or guest quarters. Additionally, the neighborhood of Pineview Village offers a relaxed and inviting atmosphere with its tree-lined streets, historic homes, and local boutiques.

For those looking for a more serene and laid-back atmosphere, the second listing in Lakeview Heights would be a great option. This cozy 2-bedroom, 1-bathroom cottage offers panoramic views of the lake and a spacious deck overlooking the water. The open living area with a fireplace creates a warm and inviting atmosphere, perfect for relaxation or entertaining. The neighborhood of Lakeview Heights is known for its tranq

#### Requiring external similarity search

###### -Load_qa_chain

###### -LLMChain

In [19]:
similar_docs = db.similarity_search(query, k=2)

In [20]:
from langchain.chains.question_answering import load_qa_chain
# 3rd method
prompt = PromptTemplate(template="{context}\n Context: {query}", 
                        input_variables = ["query", "context"]
                       )
chain = load_qa_chain(llm = llm, 
                      prompt = prompt, 
                      chain_type="stuff")
print(chain.run(input_documents = similar_docs, query=query))

/home/student/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




Based on the user's preferences for a peaceful and charming neighborhood, the two suggested listings are in Pineview Village and Lakeview Heights. Both neighborhoods offer a serene and inviting atmosphere, perfect for those seeking a retreat from city life.

The first listing in Pineview Village is a charming cottage with a cozy living room, private patio, and landscaped yard. The neighborhood is known for its tree-lined streets, historic homes, and local boutiques, creating a quaint and picturesque setting. The detached studio in the backyard also offers the potential for a home office or guest quarters, adding to the appeal for those looking for a peaceful and versatile living space.

The second listing in Lakeview Heights offers panoramic views of the lake and a spacious deck overlooking the water. The open living area with a fireplace creates a warm and inviting atmosphere, perfect for relaxation or entertaining. The neighborhood is known for its tranquil and laid-back atmosphere

In [21]:
# 4th method
chain = LLMChain(llm = llm, 
                 prompt = prompt)
result = chain.invoke(
            {"context": similar_docs, 
              "query": query},
            )["text"]
print(result)

/home/student/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(





Based on your preferences, we have found two apartments that we believe would be a perfect fit for you. The first option is a charming 2-bedroom cottage located in Pineview Village. Priced at $600,000, this home offers a cozy living room with a stone fireplace, hardwood floors, and French doors leading to a private patio. The kitchen features quartz countertops, stainless steel appliances, and a breakfast nook overlooking the landscaped yard. The outdoor space includes a fire pit, garden beds, and a detached studio, perfect for a home office or guest quarters. This home is a peaceful oasis for those seeking a retreat in a quaint village setting.

The second option is a cozy 2-bedroom cottage in Lakeview Heights, priced at $650,000. This home offers panoramic views of the lake and a spacious deck overlooking the water, perfect for morning coffee or evening sunsets. The open living area with a fireplace creates a warm and inviting atmosphere, ideal for relaxation or entertaining. With

### Create contextual memory from user preferences starting from Agent questions

In [22]:
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from typing import Dict, Any
    
history = ChatMessageHistory()
    
for i in range(len(questions)):
    history.add_ai_message(questions[i])
    history.add_user_message(answers[i])
    
# memory.save_context(history)

In [23]:
#class MementoBufferMemory(ConversationBufferMemory):
#    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
#        input_str, output_str = self._get_input_output(inputs, outputs)
#        self.chat_memory.add_ai_message(output_str)
    
#conversational_memory = MementoBufferMemory(
#                                            chat_memory=history,
#                                            memory_key = "questions_and_answers",
#                                            input_key="query")

### Create template

In [24]:
# Analyze history structure
history.messages

[AIMessage(content='How big do you want your house to be?What are 3 most important things for you in choosing this property?'),
 HumanMessage(content='A comfortable three-bedroom house with a spacious kitchen and a cozy living room.'),
 AIMessage(content='Which amenities would you like?'),
 HumanMessage(content='A quiet neighborhood, good local schools, and convenient shopping options.'),
 AIMessage(content='Which transportation options are important to you?'),
 HumanMessage(content='A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.'),
 AIMessage(content='How urban do you want your neighborhood to be?'),
 HumanMessage(content='Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.')]

In [25]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
# Parse it
def parse_messages(messages):
    parsed_content = []
    for message in messages:
        if isinstance(message, HumanMessage):
            parsed_content.append(f"Human: {message.content}")
        elif isinstance(message, AIMessage):
            parsed_content.append(f"AI: {message.content}")
    return parsed_content

formatted_messages = parse_messages(history.messages)

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import create_history_aware_retriever

# Define the updated prompt template using 'input' and 'context'
RECOMMENDER_TEMPLATE = """
    "The following is a conversation between a human and a AI Real Estate Agent"
    "You understand the preferences of the user and return the best option among the available listings"
    "If you don't know the answer related to house recommendation, say No matches are found in the available listings"
    "For other types of questions, if they are contextual to real estate, you can answer"
    "If the users ask off-topic questions out of real estate topic, say I don't know"
    "Use three sentences maximum and keep the answer concise."
    
    Personal Questions and Answers:
    {context}

    Human: {input}
    AI:
    """
    
# Update prompt template
prompt_template = PromptTemplate(
    template=RECOMMENDER_TEMPLATE,
    input_variables=["input", "context"]
)

def recommended_listings(query, formatted_messages):
    """The function returns two listings among available listings that are best suited for the user"""
    
    # Format messages into a single string to match 'context' variable
    formatted_messages_str = "\n".join(formatted_messages)
    
    # Istantiate retriever
    retriever = db.as_retriever(
                            search_type = 'similarity_score_threshold',
                            search_kwargs = {
                                            "k": 2,
                                            "score_threshold": 0.1
                            },
                           )
    
    # Create the history-aware retriever
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, prompt_template
    )
    
    # Create chain for question answering
    question_answer_chain = create_stuff_documents_chain(llm, prompt_template)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    # Prepare input dictionary for the chain
    input_data = {
        "input": query,
        "context": formatted_messages_str
    }
    
    # Invoke the chain with the provided query and formatted messages
    return rag_chain.invoke(input_data)

# Prepare formatted messages and query
formatted_messages = parse_messages(history.messages)
query = "Find two apartments based on user preferences and motivate each answer"

# Get recommendations
recommendations = recommended_listings(query, formatted_messages)

In [27]:
details = []
for k in range(len(recommendations['context'])):
    details.append(recommendations['context'][k].page_content)
details = ''.join(details)
print(recommendations['answer'] + "\n\nHere the details:\n" + details)


1. "Based on your preferences, I have found a charming 2-bedroom cottage in Pineview Village for $600,000. This cozy home features a stone fireplace, hardwood floors, and a private patio, perfect for enjoying the peaceful atmosphere of the village. With its quaint community and local boutiques, Pineview Village offers a unique and inviting lifestyle for residents."

2. "I have also found a beautiful 2-bedroom cottage in Lakeview Heights for $650,000. This home boasts panoramic views of the lake and a spacious deck, perfect for enjoying the serene surroundings. With its access to water activities and hiking trails, Lakeview Heights offers a tranquil and laid-back lifestyle for those seeking a peaceful retreat from city life."

Here the details:

Neighborhood: Pineview Village
Price: $600,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,500 sqft

Description: Step into this charming 2-bedroom, 1-bathroom cottage in Pineview Village. The cozy living room features a stone fireplace, hardwood fl